# Analysis

The main thing we'll go through today is analysis of the Demo Experiment data. We have data from 14 participants who completed the German version of the experiment.

**Before we can start:**

**a**) [Download the data](demo_data.zip) as a `.zip` file.

**b**) Extract the data from the `.zip` file to a folder that makes sense for you. The data should be stored in that location in a folder called `data`.

**c**) Start a new R script in RStudio.

Once everyone has completed these steps, we'll begin...

## 1: Load the Relevant Libraries

These are the libraries we'll use for the analysis. Each library has a comment explaining what we will use it for.

In [ ]:
options(repr.plot.width=3.5, repr.plot.height=3)

In [ ]:
library(readr)    # for reading the data into R
library(purrr)    # for easily importing multiple files
library(dplyr)    # for wrangling data (e.g., adding/renaming columns)
library(tidyr)    # for switching between long and wide formats of data
library(ordinal)  # for fitting CLMMs
library(lme4)     # for fitting LMMs

library(ggplot2)  # for visualising data
theme_set(theme_bw())

## 2: Import the Data

Let's import the data extracted from the `.zip` folder:

In [ ]:
# first, get a list of paths to all .csv data files
data_paths <- list.files("data", pattern=".*\\.csv$", full.names=TRUE)

# now, iterate over these with `read_csv()` to import them
# (note: col_types just says that we want these two columns to be stored as text, not numbers)
raw_data <- map_df(data_paths, read_csv, col_types=c(participant="c", frameRate="c"))

Let's have a look at the first few rows of the data:

In [ ]:
# print first 5 rows
head(raw_data, 5)

## 3: Format the Data

Here we make two quick changes:

1) We rename the column containing response variable to a short name: `resp`

2) We create a new column, which will be the same variable but with the order of the responses hardcoded as a factor

We store the resulting data frame in a new variable with a handy short name, `d`

In [ ]:
d <- raw_data |>
  # 1) rename to a shorter name
  rename(resp = resp_scale.response) |>
  # 2) set the factor levels explicitly
  mutate( resp_fct = factor(resp, levels=1:5, ordered=TRUE) )

Coding the variable as a factor in this last step is useful for the CLMM approach. It ensures that R knows the order of our Likert responses.

## 4: Clean the Data

To remove outliers, we may want to filter by response times, or exclude participants who only provided one response for the whole experiment.

Here are the rules we'll use:

* *Rule a)* We will exclude participants who only pressed one button for the whole experiment

* *Rule b)* We will exclude trials where participants were too fast to be paying attention (faster than 500 ms).

* *Rule c)* We will exclude trials where participants were too slow to be paying attention (slower than 15 seconds).

Remember that for your actual experiment, you will need to preregister these criteria.

<br>
We'll start by excluding participants who gave too few unique responses. To do this we first count the number of unique responses given by each participant. Then we can take from this a list of participants who only pressed one button.

In [ ]:
# get the number of unique responses from each participant
participant_variety <- d |>
  # select the relevant columns
  select(participant, resp_fct) |>
  # get unique responses
  distinct() |>
  # count unique responses per participant
  count(participant) |>
  # sort ascendingly
  arrange(n)

# get IDs of participants who only ever pressed one button
bad_participants <- participant_variety |>
  filter(n==1) |>
  pull(participant)

Then we can filter our data to not include these bad participants. This code says that we should only keep participants who are not (`!`) in (`%in%`) the vector containing bad participant IDs (`bad_participants`).

In [ ]:
# a) exclude participants who only ever pressed one button
d_clean_a <- filter(d, ! participant %in% bad_participants)

Excluding participants by Response Times (RTs) is much easier. We can say we want to keep trials where participants were under the maximum RT, and above the minimum RT.

In [ ]:
# b) exclude trials that were too slow
d_clean_b <- filter(d_clean_a, resp_scale.rt < 15)

# c) exclude trials that were too fast
d_clean_c <- filter(d_clean_b, resp_scale.rt > 0.5)

The last step is to calculate the number of participants/trials lost at each step. We can do this with the `nrow()` function to count the number of rows (i.e., trials) in the dataframe. To count the number of bad participants, we can use `length()`.

In [ ]:
# there were 0 participants excluded on the basis of only ever pressing one button
length(bad_participants)

In [ ]:
# there were 14 trials excluded on the basis of slow responses
nrow(d_clean_a) - nrow(d_clean_b)

In [ ]:
# there were 0 trials excluded on the basis of fast responses
nrow(d_clean_b) - nrow(d_clean_c)

It looks like the data was quite good quality overall! 💪

## 5: Fit the Models!

Now that the data are formatted and cleaned we can fit three models to estimate our norms:

1. The traditional model (LM; `lm()`), where each item's ratings are averaged.

2. A linear mixed-effects model (LMM; `lmer()`), where we norm via random effects, but still treat the ratings as continuous.

3. A cumulative-link mixed-effects model (CLMM; `clmm()`), where we norm via random effects, on a latent normal distribution.

### LM

First, the traditional approach is to take a raw average of all ratings for each item (a simple linear model). A simple way of estimating this is:

In [ ]:
# estimate a raw average for each item
est_lm <- d_clean_c |>
  group_by(word) |>
  summarise(M = mean(resp)) |>
  # store the model type as a variable (we'll use this later)
  mutate(model = "LM")

# print first 5 rows
head(est_lm, 5)

It might not feel like it, but this is already a model (albeit a very simple one). A way of fitting this with the `lm()` function would be:

In [ ]:
m_lm <- lm(resp ~ 0 + word, data = d_clean_c)

If you want to convince yourself that this is the same thing, then compare the output of `m_lm` with the estimates in `est_lm`. You should see exactly the same numbers!

In [ ]:
m_lm

### LMM

To norm via random effects in an LMM, we can use the `lmer()` function. We include random (varying) intercepts for participants (`(1 | participant)`) and items (`(1 | item)`). We will only extract estimates for items, but we also include random intercepts for participants because it usually improves the accuracy of the estimates for items.

In [ ]:
# Fit LMM
m_lmm <- lmer(
  resp ~ 1 + (1 | word) + (1 | participant),
  data = d_clean_c
)

# print result
m_lmm

This tells us that the average rating overall (the `Intercept`) was 3.42. The standard deviations of the random effects tell us that words (`1.1517`) were more variable than participants (`0.2165`).

We can now extract the estimated norms for each word.

In [ ]:
# extract estimates (item random effects)
est_lmm <- ranef(m_lmm)$word |>
  # put into a tidy dataframe
  as_tibble(rownames = "word") |>
  # M, standing for mean
  rename(M = `(Intercept)`) |>
  # store the model type as a variable (we'll use this later)
  mutate(model = "LMM")

# print first 5 rows
head(est_lmm, 5)

### CLMM

To norm via random effects in a CLMM, we can use the `clmm()` function. We include the same random effects structure as above. The key difference is that now we are using a model more appropriate for ordinal ratings.

In [ ]:
# Fit CLMM (probit-link for comparability)
m_clmm <- clmm(
  resp_fct ~ 1 + (1 | word) + (1 | participant),
  data = d_clean_c,
  link = "probit"
)

# print result
m_clmm

This gives us similar information to that seen above, but also gives us the locations of the thresholds on the latent distribution. For example, the threshold between the Likert responses of *3* and *4* (or, `3|4`) is at `-0.1341`.

As with the LMM, we can extract norms for words from the CLMM like so:

In [ ]:
# extract estimates (item random effects)
est_clmm <- ranef(m_clmm)$word |>
  # put into a tidy dataframe
  as_tibble(rownames = "word") |>
  # M, standing for mean
  rename(M = `(Intercept)`) |>
  # store the model type as a variable
  mutate(model = "CLMM")

# print first 5 rows
head(est_clmm, 5)

## 6: Compare the AICs

One nice comparison we can do is on the Akaike Information Criteria (AIC). This is something we touched on in the session on [Empirical Research](https://jackedtaylor.github.io/expra-wise23/introduction/empirical_research.html). The AIC, and estimators like it, can help us decide which model is most appropriate for the trade-off between model complexity and model likelihood.

In [ ]:
AIC(m_lm)

In [ ]:
AIC(m_lmm)

In [ ]:
AIC(m_clmm)

This shows that the CLMM has the lowest AIC of the three models, and so is likely to be the most appropriate for the data.

## 7: Join the Estimates Together

Now that we have the estimates, we can join these rows together into one big dataframe, `norms`.

In [ ]:
# Stick the dataframes together, end-to-end
norms <- bind_rows(est_lm, est_lmm, est_clmm)

# print first 5 rows
head(norms, 5)

We may also want the data in wide format (different model estimates are in different columns). This is possible with the `pivot_wider()` function. This lets us compare the estimates for each word from the three approaches.

In [ ]:
# pivot the data into wide format
norms_wide <- pivot_wider(norms, names_from=model, values_from=M)

# print first 5 rows
head(norms_wide, 5)

## 8: Examine Correlations

We may be interested in how similar our estimates are from our three approaches. The variables are all very highly correlated because they are all trying to capture very similar information. However, the CLMM approach is least like the other two (lower Pearson's r values in the matrix).

In [ ]:
norms_wide |>
  # remove non-numeric column
  select(-word) |>
  # get correlation matrix
  cor()

We can also visualise these correlations with `ggplot2`

First, the correlations between the LM and LMM approach are very linear.

In [ ]:
norms_wide |>
  ggplot(aes(LMM, LM)) +
  geom_point() +
  geom_smooth(method="lm")

However, the correlation between the CLMM and LMM estimates is very nonlinear. This reflects that the LM estimates are distorted by the LM method incorrectly treating the ordinal scale as though it is an interval scale.

In [ ]:
norms_wide |>
  ggplot(aes(CLMM, LM)) +
  geom_point() +
  geom_smooth(method="lm")

## Bonus 1: Recreate the Plot in Pollock (2018)

Let's see whether we can recreate the plot that Pollock ([2018](https://doi.org/10.3758/s13428-017-0938-y)) made of the data from Brysbaert et al. ([2014](https://doi.org/10.3758/s13428-013-0403-5)).

In [ ]:
# first, calculate the means and SDs of raw ratings
raw_summ <- d_clean_c |>
  group_by(word) |>
  summarise(M=mean(resp), SD=sd(resp))

# then, show the Ms and SDs in a scatter plot
raw_summ |>
  ggplot(aes(M, SD)) +
  geom_point()

Even our small sample size shows a very similar pattern to that described by Pollock.